In [ ]:
!pip3 install boto3

In [ ]:
import boto3

session=boto3.Session(
    aws_access_key_id='',   
    aws_secret_access_key='',
    region_name='ap-south-1'        
)

ec2 = session.resource('ec2')

In [ ]:
instance = ec2.create_instances(
    ImageId='ami-0f5ee92e2d63afc18',  
    InstanceType='t2.micro',
    MinCount=1,
    MaxCount=1,
    SecurityGroups=['ngo-donation-module'],  
    KeyName='NGO',
    UserData=open('script.sh', 'r').read(),
)[0]

instance.create_tags(Tags=[{'Key': 'Name', 'Value': 'Ngo-Payment'}])

print(instance)
print('Running and waiting for DNS')
instance.wait_until_running()


public_dns = instance.public_dns_name
print("Public DNS:", public_dns)

# RDS Config

In [ ]:
client=session.client('rds')
db_instance_identifier='ngo-donation'

In [ ]:
def create_db_inst():
    
    response=client.create_db_instance(
        DBInstanceIdentifier = db_instance_identifier,
        DBInstanceClass = 'db.t3.micro',
        Engine = 'mysql',
        MasterUsername = 'admin',
        VpcSecurityGroupIds=['sg-0c2c2a3c1b9891a6c'],
        MasterUserPassword = 'Rds-2023',
        AllocatedStorage = 20,  
        AvailabilityZone = 'ap-south-1a',  
        DBParameterGroupName = 'default.mysql8.0',  
        MultiAZ = False,
        Tags=[{'Key': 'Name', 'Value': 'MySqlDBInstance'}], 
    )

    return response

In [ ]:
response=create_db_inst()
print(response)

client.get_waiter('db_instance_available').wait(DBInstanceIdentifier=db_instance_identifier)

print("DB instance created successfully!")
print("DB Instance ID:", response['DBInstance']['DBInstanceIdentifier'])